# 05e - Vertex AI > Training > Training Pipelines - With Python Source Distribution

### 05 Series Overview
Where a model gets trained is where it consumes computing resources.  With Vertex AI, you have choices for configuring the computing resources available at training.  This notebook is an example of an execution environment.  When it was set up there were choices for machine type and accelerators (GPUs).  

In the `05` notebook, the model training happened directly in the notebook.  The models were then imported to Vertex AI and deployed to an endpoint for online predictions. 

In this `05a-05i` series of demonstrations, the same model is trained using managed computing resources in Vertex AI as custom training jobs.  These jobs will be demonstrated as:

-  Custom Job from a python script (`05a`), python source distribution (`05b`), and custom container (`05c`)
-  Training Pipeline that trains and saves models from a python script (`05d`), python source distribution (`05e`), and custom container (`05f`)
-  Hyperparameter Tuning Jobs from a python script (`05g`), python source distribution (`05h`), and custom container (`05i`)

### This Notebook (`05e`): An extension of `05b` as a Training Pipeline that saves the model to Vertex AI > Models 
This notebook trains the same Tensorflow Keras model from `05` by first modifying and saving the training code to a python script (same as `05a`).  Then a Python source distribution is built containing the script.  While this example fits nicely in a single script, larger examples will benefit from the flexibility offered by source distributions and this notebook gives an example of making the shift. 

The source distribution is then used as an input for a Vertex AI > Training > Training Pipeline that is also assigned compute resources and a [pre-built container for custom training](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers) for executing the training in a managed service.  This is done with the [Vertex AI Python SDK](https://googleapis.dev/python/aiplatform/latest/aiplatform.html#) using the class [`aiplatform.CustomPythonPackageTrainingJob()`](https://googleapis.dev/python/aiplatform/latest/aiplatform.html#google.cloud.aiplatform.CustomPythonPackageTrainingJob).

The client used here is `aiplatform.CustomPythonPackageTrainingJob(python_package_gcs_uri=)` which is very similar to what was used for the source distribution in `05b` `aiplatform.CustomJob()`. The functional difference is that this method automatically uploads the final saved model to Vertex AI > Models.  Running the job this way first triggers a job in Vertex AI > Training > Training Pipeline.  This Training Pipeline triggers a Custom Job in Vertex AI > Training > Custom Jobs.  If The Custom Job completes successfully then the final saved model is registered in Vertex AI > Models.

The training can be reviewed with Vertex AI's managed Tensorboard under Vertex AI > Experiments > Experiments, or by clicking on the `05e...` custom job under Vertex AI > Training > Custom Jobs and then clicking the 'Open Tensorboard' link. 

<img src="architectures/overview/Training.png">

### Prerequisites:
-  01 - BigQuery - Table Data Source
-  Understanding:
    -  05 - Vertex AI > Notebooks - Models Built in Notebooks with Tensorflow
        -  Contains a more granular review of the Tensorflow model training

### Overview:
-  Setup
-  Connect to Tensorboard instance from 05
-  Create a `train.py` Python script that recreates the local training in 05
-  Build a Python source distribution that contains the `train.py` script
-  Use Python Client google.cloud.aiplatform for Vertex AI
   -  Custom training job with aiplatform.CustomPythonPackageTrainingJob
      -  Run job with .run
   -  Create Endpoint with Vertex AI with aiplatform.Endpoint.create
      -  Deploy model to endpoint with .deploy 
-  Online Prediction demonstrated using Vertex AI Endpoint with deployed model
   -  Get records to score from BigQuery table
   -  Prediction with aiplatform.Endpoint.predict
   -  Prediction with REST
   -  Prediction with gcloud (CLI)

### Resources:
-  [BigQuery Tensorflow Reader](https://www.tensorflow.org/io/tutorials/bigquery)
-  [Keras Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential)
   -  [Keras API](https://www.tensorflow.org/api_docs/python/tf/keras)
-  [Python Client For Google BigQuery](https://googleapis.dev/python/bigquery/latest/index.html)
-  [Tensorflow Python Client](https://www.tensorflow.org/api_docs/python/tf)
-  [Tensorflow I/O Python Client](https://www.tensorflow.org/io/api_docs/python/tfio/bigquery)
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)
-  [Create a Python source distribution](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container) for a Vertex AI custom training job
-  Containers for training (Pre-Built)
   -  [Overview](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container)
   -  [List](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)


---
## Vertex AI - Conceptual Flow

<img src="architectures/slides/05e_arch.png">

---
## Vertex AI - Workflow

<img src="architectures/slides/05e_console.png">

---
## Setup

inputs:

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [2]:
REGION = 'us-central1'
DATANAME = 'fraud'
NOTEBOOK = '05e'

# Resources
TRAIN_IMAGE = 'us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-7:latest'
DEPLOY_IMAGE ='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest'
TRAIN_COMPUTE = 'n1-standard-4'
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters
EPOCHS = 10
BATCH_SIZE = 100

packages:

In [3]:
from google.cloud import aiplatform
from datetime import datetime

from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

clients:

In [4]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bigquery = bigquery.Client()

parameters:

In [5]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{DATANAME}/models/{NOTEBOOK}"
DIR = f"temp/{NOTEBOOK}"

In [6]:
# Give service account roles/storage.objectAdmin permissions
# Console > IMA > Select Account <projectnumber>-compute@developer.gserviceaccount.com > edit - give role
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'1026793852137-compute@developer.gserviceaccount.com'

environment:

In [7]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Get Vertex AI Experiments Tensorboard Instance Name
[Vertex AI Experiments](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview) has managed [Tensorboard](https://www.tensorflow.org/tensorboard) instances that you can track Tensorboard Experiments (a training run or hyperparameter tuning sweep).  

The training job will show up as an experiment for the Tensorboard instance and have the same name as the training job ID.

This code checks to see if a Tensorboard Instance has been created in the project, retrieves it if so, creates it otherwise:

In [8]:
tb = aiplatform.Tensorboard.list(filter=f'display_name={DATANAME}')
if tb:
    tb = tb[0]
else:
    tb = aiplatform.Tensorboard.create(display_name = DATANAME, labels = {'notebook':f'{DATANAME}'})

In [9]:
tb.resource_name

'projects/1026793852137/locations/us-central1/tensorboards/1212224763762573312'

---
## Training

### Assemble Python File for Training

Create the main python trainer file as `/train.py`:

In [10]:
!mkdir -p {DIR}/source/trainer

In [11]:
%%writefile {DIR}/source/trainer/train.py

# package import
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
import tensorflow as tf
from google.cloud import bigquery
import argparse
import os
import sys

# import argument to local variables
parser = argparse.ArgumentParser()
# the passed param, dest: a name for the param, default: if absent fetch this param from the OS, type: type to convert to, help: description of argument
parser.add_argument('--epochs', dest = 'epochs', default = 10, type = int, help = 'Number of Epochs')
parser.add_argument('--batch_size', dest = 'batch_size', default = 32, type = int, help = 'Batch Size')
parser.add_argument('--var_target', dest = 'var_target', type=str)
parser.add_argument('--var_omit', dest = 'var_omit', type=str, nargs='*')
parser.add_argument('--project_id', dest = 'project_id', type=str)
parser.add_argument('--dataname', dest = 'dataname', type=str)
parser.add_argument('--region', dest = 'region', type=str)
parser.add_argument('--notebook', dest = 'notebook', type=str)
args = parser.parse_args()

# built in parameters for data source:
PROJECT_ID = args.project_id
DATANAME = args.dataname
REGION = args.region
NOTEBOOK = args.notebook

# clients
bigquery = bigquery.Client(project = PROJECT_ID)

# get schema from bigquery source
query = f"SELECT * FROM {DATANAME}.INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{DATANAME}_prepped'"
schema = bigquery.query(query).to_dataframe()

# get number of classes from bigquery source
nclasses = bigquery.query(query = f'SELECT DISTINCT {args.var_target} FROM {DATANAME}.{DATANAME}_prepped WHERE {args.var_target} is not null').to_dataframe()
nclasses = nclasses.shape[0]

# Make a list of columns to omit
OMIT = args.var_omit + ['splits']

# use schema to prepare a list of columns to read from BigQuery
selected_fields = schema[~schema.column_name.isin(OMIT)].column_name.tolist()

# all the columns in this data source are either float64 or int64
output_types = [dtypes.float64 if x=='FLOAT64' else dtypes.int64 for x in schema[~schema.column_name.isin(OMIT)].data_type.tolist()]

# remap input data to Tensorflow inputs of features and target
def transTable(row_dict):
    target=row_dict.pop(args.var_target)
    target = tf.one_hot(tf.cast(target,tf.int64), nclasses)
    target = tf.cast(target, tf.float32)
    return(row_dict, target)

# function to setup a bigquery reader with Tensorflow I/O
def bq_reader(split):
    reader = BigQueryClient()

    training = reader.read_session(
        parent = f"projects/{PROJECT_ID}",
        project_id = PROJECT_ID,
        table_id = f"{DATANAME}_prepped",
        dataset_id = DATANAME,
        selected_fields = selected_fields,
        output_types = output_types,
        row_restriction = f"splits='{split}'",
        requested_streams = 3
    )
    
    return training

train = bq_reader('TRAIN').parallel_read_rows().prefetch(1).map(transTable).shuffle(args.batch_size*10).batch(args.batch_size)
validate = bq_reader('VALIDATE').parallel_read_rows().prefetch(1).map(transTable).batch(args.batch_size)
test = bq_reader('TEST').parallel_read_rows().prefetch(1).map(transTable).batch(args.batch_size)

# Logistic Regression

# model input definitions
feature_columns = {header: tf.feature_column.numeric_column(header) for header in selected_fields if header != args.var_target}
feature_layer_inputs = {header: tf.keras.layers.Input(shape = (1,), name = header) for header in selected_fields if header != args.var_target}

# feature columns to a Dense Feature Layer
feature_layer_outputs = tf.keras.layers.DenseFeatures(feature_columns.values())(feature_layer_inputs)

# batch normalization then Dense with softmax activation to nclasses
layers = tf.keras.layers.BatchNormalization()(feature_layer_outputs)
layers = tf.keras.layers.Dense(nclasses, activation = tf.nn.softmax)(layers)

# the model
model = tf.keras.Model(
    inputs = feature_layer_inputs,
    outputs = layers
)
opt = tf.keras.optimizers.SGD() #SGD or Adam
loss = tf.keras.losses.CategoricalCrossentropy()
model.compile(
    optimizer = opt,
    loss = loss,
    metrics = ['accuracy', tf.keras.metrics.AUC(curve='PR')]
)

# setup tensorboard logs and train
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=os.environ['AIP_TENSORBOARD_LOG_DIR'], histogram_freq=1)
history = model.fit(train, epochs = args.epochs, callbacks = [tensorboard_callback], validation_data = validate)

# output the model save files
model.save(os.getenv("AIP_MODEL_DIR"))

Writing temp/05e/source/trainer/train.py


### Assemble Python Source Distribution

create `setup.py` file:

In [12]:
%%writefile {DIR}/source/setup.py
from setuptools import setup
from setuptools import find_packages

REQUIRED_PACKAGES = ['tensorflow_io']

setup(
    name = 'trainer',
    version = '0.1',
    install_requires = REQUIRED_PACKAGES, 
    packages = find_packages(),
    include_package_data = True,
    description='Training Package'
)

Writing temp/05e/source/setup.py


add `__init__.py` file to the trainer modules folder:

In [13]:
!touch {DIR}/source/trainer/__init__.py

Create the source distribution and copy it to the projects storage bucket:
- change to the local direcory with the source folder
- remove any previous distributions
- tar and gzip the source folder
- copy the distribution to the project folder on GCS
- change back to the local project directory

In [14]:
%cd {DIR}

!rm -f source.tar source.tar.gz
!tar cvf source.tar source
!gzip source.tar
!gsutil cp source.tar.gz {URI}/{TIMESTAMP}/source.tar.gz

temp = '../'*(DIR.count('/')+1)
%cd {temp}

/home/jupyter/vertex-ai-mlops/temp/05e
source/
source/setup.py
source/trainer/
source/trainer/__init__.py
source/trainer/train.py
Copying file://source.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  2.1 KiB/  2.1 KiB]                                                
Operation completed over 1 objects/2.1 KiB.                                      
/home/jupyter/vertex-ai-mlops


### Setup Training Job

In [15]:
CMDARGS = [
    "--epochs=" + str(EPOCHS),
    "--batch_size=" + str(BATCH_SIZE),
    "--var_target=" + VAR_TARGET,
    "--var_omit=" + VAR_OMIT,
    "--project_id=" + PROJECT_ID,
    "--dataname=" + DATANAME,
    "--region=" + REGION,
    "--notebook=" + NOTEBOOK
]

In [16]:
customJob = aiplatform.CustomPythonPackageTrainingJob(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    python_package_gcs_uri = f"{URI}/{TIMESTAMP}/source.tar.gz",
    python_module_name = "trainer.train",
    container_uri = TRAIN_IMAGE,
    model_serving_container_image_uri = DEPLOY_IMAGE,
    staging_bucket = f"{URI}/{TIMESTAMP}",
    labels = {'notebook':f'{NOTEBOOK}'}
)

### Run Training Job

In [17]:
model = customJob.run(
    model_display_name = f'{NOTEBOOK}_{DATANAME}',
    model_labels = {'notebook':f'{NOTEBOOK}'},
    model_id = f'model_{NOTEBOOK}_{DATANAME}',
    is_default_version = True,
    model_version_aliases = [f'time-{TIMESTAMP}'],
    model_version_description = f'time-{TIMESTAMP}',
    base_output_dir = f"{URI}/{TIMESTAMP}",
    service_account = SERVICE_ACCOUNT,
    args = CMDARGS,
    replica_count = 1,
    machine_type = TRAIN_COMPUTE,
    accelerator_count = 0,
    tensorboard = tb.resource_name
)

Training Output directory:
gs://statmike-mlops-349915/fraud/models/05e/20220728002203 
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/3427571643818442752?project=1026793852137
CustomPythonPackageTrainingJob projects/1026793852137/locations/us-central1/trainingPipelines/3427571643818442752 current state:
PipelineState.PIPELINE_STATE_RUNNING
View backing custom job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/435149593756303360?project=1026793852137
View tensorboard:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+1212224763762573312+experiments+435149593756303360
CustomPythonPackageTrainingJob projects/1026793852137/locations/us-central1/trainingPipelines/3427571643818442752 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomPythonPackageTrainingJob projects/1026793852137/locations/us-central1/trainingPipelines/3427571643818442752 

In [18]:
model.display_name

'05e_fraud'

---
## Serving

### Create An Endpoint

In [35]:
SERIES = ''.join(filter(str.isdigit, NOTEBOOK))
endpoints = aiplatform.Endpoint.list(filter = f"display_name={SERIES}_{DATANAME}")
if endpoints:
    endpoint = endpoints[0]
    print(f"Endpoint Exists: {endpoints[0].resource_name}")
else:
    endpoint = aiplatform.Endpoint.create(
        display_name = f"{SERIES}_{DATANAME}",
        labels = {'notebook':f"{SERIES}"}    
    )
    print(f"Endpoint Created: {endpoint.resource_name}")

Endpoint Exists: projects/1026793852137/locations/us-central1/endpoints/7252545822577917952


In [36]:
endpoint.display_name

'05_fraud'

### Deploy Model To Endpoint

In [37]:
dmodels = endpoint.list_models()

check = 0
if dmodels:
    for dmodel in dmodels:
        if dmodel.model == model.resource_name and dmodel.model_version_id == model.version_id and dmodel.id in endpoint.traffic_split:
            print(f'This model (and version) already deployed with {endpoint.traffic_split[dmodel.id]}% of traffic')
            check = 1
    
if check == 0:
    print(f'Deploying model with 100% of traffic...')
    endpoint.deploy(
        model = model,
        deployed_model_display_name = f'{NOTEBOOK}_{DATANAME}',
        traffic_percentage = 100,
        machine_type = DEPLOY_COMPUTE,
        min_replica_count = 1,
        max_replica_count = 1
    )     

Deploying model with 100% of traffic...
Deploying Model projects/1026793852137/locations/us-central1/models/model_05e_fraud to Endpoint : projects/1026793852137/locations/us-central1/endpoints/7252545822577917952
Deploy Endpoint model backing LRO: projects/1026793852137/locations/us-central1/endpoints/7252545822577917952/operations/4626216683063738368


/opt/conda/lib/python3.7/site-packages/proto/marshal/rules/enums.py:40: UserWarning: Unrecognized DeploymentResourcesType enum value: 3
  value=value,


Endpoint model deployed. Resource name: projects/1026793852137/locations/us-central1/endpoints/7252545822577917952


### Remove Deployed Models without Traffic

In [38]:
for dmodel in endpoint.list_models():
    if dmodel.id in endpoint.traffic_split:
        print(f"Model {dmodel.display_name} has traffic = {endpoint.traffic_split[dmodel.id]}")
    else:
        endpoint.undeploy(deployed_model_id = dmodel.id)
        print(f"Undeployed {dmodel.display_name} version {dmodel.model_version_id} as it has no traffic.")

Model 05e_fraud has traffic = 100
Undeploying Endpoint model: projects/1026793852137/locations/us-central1/endpoints/7252545822577917952
Undeploy Endpoint model backing LRO: projects/1026793852137/locations/us-central1/endpoints/7252545822577917952/operations/6544750124323569664
Endpoint model undeployed. Resource name: projects/1026793852137/locations/us-central1/endpoints/7252545822577917952
Undeployed 05d_fraud version 1 as it has no traffic.


In [39]:
endpoint.traffic_split

{'159635894213541888': 100}

In [40]:
endpoint.list_models()

[id: "159635894213541888"
 model: "projects/1026793852137/locations/us-central1/models/model_05e_fraud"
 display_name: "05e_fraud"
 create_time {
   seconds: 1659091080
   nanos: 303190000
 }
 dedicated_resources {
   machine_spec {
     machine_type: "n1-standard-4"
   }
   min_replica_count: 1
   max_replica_count: 1
 }
 model_version_id: "1"]

---
## Prediction

### Prepare a record for prediction: instance and parameters lists

In [41]:
pred = bigquery.query(query = f"SELECT * FROM {DATANAME}.{DATANAME}_prepped WHERE splits='TEST' LIMIT 10").to_dataframe()

In [42]:
pred.head(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,32799,1.153477,-0.047859,1.358363,1.480620,-1.222598,-0.481690,-0.654461,0.128115,0.907095,...,-0.025964,0.701843,0.417245,-0.257691,0.060115,0.035332,0.0,0,e9d16028-4b41-4753-87ee-041d33642ae9,TEST
1,35483,1.286640,0.072917,0.212182,-0.269732,-0.283961,-0.663306,-0.016385,-0.120297,-0.135962,...,0.052674,0.076792,0.209208,0.847617,-0.086559,-0.008262,0.0,0,8b319d3a-2b2d-445b-a9a2-0da3d664ec2a,TEST
2,163935,1.961967,-0.247295,-1.751841,-0.268689,0.956431,0.707211,0.020675,0.189433,0.455055,...,0.186420,-1.621368,-0.131098,0.034276,-0.004909,-0.090859,0.0,0,788afb87-60aa-4482-8b48-c924bec634aa,TEST
3,30707,-0.964364,0.176372,2.464128,2.672539,0.145676,-0.152913,-0.591983,0.305066,-0.148034,...,-0.024200,0.365226,-0.745369,-0.060544,0.095692,0.217639,0.0,0,473d0936-1974-4ae8-ab70-230e7599bd3f,TEST


In [43]:
newob = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET, 'splits'])]].to_dict(orient='records')[0]
#newob

In [44]:
instances = [json_format.ParseDict(newob, Value())]
parameters = json_format.ParseDict({}, Value())

### Get Predictions: Python Client

In [45]:
prediction = endpoint.predict(instances=instances, parameters=parameters)
prediction

Prediction(predictions=[[0.999707639, 0.000292325]], deployed_model_id='159635894213541888', model_version_id='', model_resource_name='projects/1026793852137/locations/us-central1/models/model_05e_fraud', explanations=None)

In [46]:
prediction.predictions[0]

[0.999707639, 0.000292325]

In [47]:
np.argmax(prediction.predictions[0])

0

### Get Predictions: REST

In [48]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [49]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    [
      0.999707639,
      0.000292325
    ]
  ],
  "deployedModelId": "159635894213541888",
  "model": "projects/1026793852137/locations/us-central1/models/model_05e_fraud",
  "modelDisplayName": "05e_fraud"
}


### Get Predictions: gcloud (CLI)

In [50]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[[0.999707639, 0.000292325]]


---
## Remove Resources
see notebook "99 - Cleanup"